In [18]:
import pandas as pd
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns 
import math
import random
sns.set(font_scale=1.56)
# from eli5.sklearn import PermutationImportance
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from mlxtend.regressor import StackingRegressor
from mlxtend.data import boston_housing_data
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold, learning_curve, train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import RFECV
from sklearn import preprocessing
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
# for display dataframe
from IPython.display import display
from IPython.display import display_html
import warnings

warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
%matplotlib inline

In [19]:
weight = pd.read_csv('./weight.csv',index_col=0) #將第一欄當名稱
machine = pd.read_csv('./machine.csv',index_col=0)

In [20]:
weight

,PEP1_weight,PEP2_weight,PEP3_weight
c1,0.90,0.5,0.9
c2,0.90,0.8,1.0
c3,0.90,0.7,0.5
c4,1.00,0.7,0.7
c5,0.95,0.9,0.9
c6,0.85,0.6,0.4
c7,1.00,0.8,0.6
c8,0.93,0.7,0.3
c9,0.94,0.9,0.8
c10,0.88,0.8,0.4


In [21]:
machine.iloc[:][0:1]

,U%,PP,MURA
V1,91,90,100


In [8]:
machine

,U%,PP,MURA
V1,91,90,100
V2,98,93,92
V3,96,95,90
V4,95,0,98
V5,99,0,99
V6,99,0,94
V7,91,93,90
V8,99,100,100
V9,98,100,97


In [65]:
class Gene:
    # 隨機產生基因染色體：隨機產生 pop_size 組，維度為 pop_dim，長度為 chrom_length 的染色體。
    # 每個染色體皆是0或是1，長度越長代表細分程度越高。    
    def geneEncoding(pop_size,chromosome_length):
        pop = []
        for i in range(pop_size):            
            temp = []
            length_sum = 0
            for j in range(0,len(chromosome_length)):
                if j == 0:
                    length_sum = length_sum
                else:
                    length_sum = length_sum + chromosome_length[j-1]
                    
                tmp_gene_sum = 0
                for a in range(length_sum,length_sum + chromosome_length[j]):
                    tmp_gene = random.randint(0, 1)
                    tmp_gene_sum = tmp_gene_sum + tmp_gene
                    if a == length_sum + chromosome_length[j]-1:
                        #最後一個
                        if tmp_gene_sum > 1:
                            #總和大於1
                            temp.append(0)
                            tmp_gene_sum = tmp_gene_sum - tmp_gene + 0
                        elif tmp_gene_sum == 1:
                            if tmp_gene == 0:
                                temp.append(0)
                                tmp_gene_sum = tmp_gene_sum - tmp_gene + 0
                            else:
                                temp.append(1)
                                tmp_gene_sum = tmp_gene_sum - tmp_gene + 1
                        elif tmp_gene_sum == 0:
                            #總和等於0
                            temp.append(1)                        
                            tmp_gene_sum = tmp_gene_sum - tmp_gene + 1
                    else:
                        #非最後一個
                        if tmp_gene_sum > 1:
                            #總和大於1
                            temp.append(0)
                            tmp_gene_sum = tmp_gene_sum - tmp_gene + 0
                        elif tmp_gene_sum == 1:
                            if tmp_gene == 0:
                                temp.append(0)
                                tmp_gene_sum = tmp_gene_sum - tmp_gene + 0
                            else:
                                temp.append(1)
                                tmp_gene_sum = tmp_gene_sum - tmp_gene + 1
                        elif tmp_gene_sum == 0:
                            #總和等於0
                            temp.append(0)
                            tmp_gene_sum = tmp_gene_sum - tmp_gene + 0
            pop.append(temp)
        return pop
    
    #計算累加機率
    def cumsum(fit_value):
        #從倒數第二個開始倒數到第0個
        for i in range(len(fit_value)-2, -1, -1):
            t = 0
            j = 0
            while(j <= i):
                t += fit_value[j]
                j += 1
            fit_value[i] = t
        fit_value[len(fit_value)-1] = 1 # 最後一個值填1
        return fit_value          
            
    
    #輪盤選擇法
    def selection(pop, fit_value, target):
        if target == 'max':
            for i in range(len(fit_value)):
                fit_value[i] = fit_value[i] - min(fit_value) + 0.001
        elif target == 'min':
            
    
    
    
    
    
    
    
    #找出最佳解與基因組合的位置，並儲存
    #挑選適應度好的，利用機率的方式讓壞的結果也可能會變成最好   
    def find_best(fit_value, pop, target):
        count = -1 
        if target == 'max':
            Best = -99999999
            for i in range(len(fit_value)):
                if Best < fit_value[i]:
                    Best = fit_value[i]
                    count = i
            if len(BestValue) == 0:
                BestValue.append(fit_value[i])
                BestGene.append(pop[i])
            else:
                if BestValue[len(BestValue) - 1] < fit_value[i]:
                    BestValue.append(fit_value[i])
                    BestGene.append(pop[i])
                else:
                    BestValue.append(BestValue[len(BestValue) - 1])
                    BestGene.append(BestGene[len(BestGene) - 1])
        elif target == 'min':
            Best = 99999999
            for i in range(len(fit_value)):
                if Best > fit_value[i]:
                    Best = fit_value[i]
                    count = i
            if len(BestValue) == 0:
                BestValue.append(fit_value[i])
                BestGene.append(pop[i])
            else:
                if BestValue[len(BestValue) - 1] > fit_value[i]:
                    BestValue.append(fit_value[i])
                    BestGene.append(pop[i])
                else:
                    BestValue.append(BestValue[len(BestValue) - 1])
                    BestGene.append(BestGene[len(BestGene) - 1])
                
            
            
        #idx = np.random.choice(np.arange(pop_size), size=pop_size, replace = True, p=fit_value / np.array(fit_value).sum())
    
    #挑選適應度好的，直接選分數最高的作為結果     
        
    
Gene.geneEncoding(pop_size, chromosome_length)  

[[1, 0, 0, 1, 0, 0, 0, 1, 0],
 [1, 0, 0, 1, 0, 0, 0, 0, 1],
 [1, 0, 0, 0, 1, 0, 0, 0, 1],
 [0, 0, 1, 1, 0, 0, 0, 0, 1],
 [0, 0, 1, 0, 1, 0, 1, 0, 0]]

In [66]:
# 在這裡定義目標式，這裡是範例：
# y = 10 * math.sin(5 * a) + 7 * math.cos(4 * b)

def calobjValue(pop,chromosome_length,machine,weight_give): 
    obj_value = []
    score = 0
    weight_list = list(weight_give)
    for i in range(len(pop)):
        score = 0
        length_sum = 0
        for j in range(0,len(chromosome_length)):
            if j == 0:
                length_sum = length_sum
            else:
                length_sum = length_sum + chromosome_length[j-1]
                
            for a in range(length_sum,length_sum + chromosome_length[j]):
                #依據組合將score算出
                if pop[i][a] == 1:
                    for b in range(machine.shape[1]):
                        score = score + machine.iloc[a][b] * weight_give.iloc[0][j]
        obj_value.append(score)
        #print(score)
    return obj_value
calobjValue(pop.copy(),chromosome_length,machine.copy(),weight_give.copy())

[618.5, 596.0, 598.5, 596.0, 596.0]

In [67]:
#參數設定

loop_times = 100 # 迭代次數
pop_size = 5 # 基因組數/母體數量
pc = 0.6 #交配機率，設0代表不進行交配
pm = 0.01 #變異機率，設0代表不進行突變

chromosome_length = [3,3,3] # 染色體長度，越大代表細分程度越高，也代表要跑越久（細分程度 = 2^chromosome_length）
target = 'max' #望大或望小
BestValue = [] #儲存最佳解
BestGene = [] #儲存最佳基因組

weight_give = weight.iloc[:][0:1]#指定產品

In [68]:
#基因演算法求最佳解
pop = Gene.geneEncoding(pop_size, chromosome_length) #產生母體

for i  in range(loop_times):
    
    #decodechromosome 將二進制轉成十進制，再轉成一定範圍
    #計算f(x)
    fit_value = calobjValue(pop.copy(),chromosome_length,machine,weight_give)
    #挑選適應度好
    Gene.find_best(fit_value, pop, target)
    #利用輪盤選擇法進行挑選複製
    newpop = Gene.selection(pop,fit_value.copy(),target)
    
    
    
    
    
    
    


In [69]:
BestGene

[[0, 0, 1, 1, 0, 0, 0, 1, 0],
 [0, 0, 1, 1, 0, 0, 0, 1, 0],
 [0, 0, 1, 1, 0, 0, 0, 1, 0],
 [0, 0, 1, 1, 0, 0, 0, 1, 0],
 [0, 0, 1, 1, 0, 0, 0, 1, 0],
 [0, 0, 1, 1, 0, 0, 0, 1, 0],
 [0, 0, 1, 1, 0, 0, 0, 1, 0],
 [0, 0, 1, 1, 0, 0, 0, 1, 0],
 [0, 0, 1, 1, 0, 0, 0, 1, 0],
 [0, 0, 1, 1, 0, 0, 0, 1, 0],
 [0, 0, 1, 1, 0, 0, 0, 1, 0],
 [0, 0, 1, 1, 0, 0, 0, 1, 0],
 [0, 0, 1, 1, 0, 0, 0, 1, 0],
 [0, 0, 1, 1, 0, 0, 0, 1, 0],
 [0, 0, 1, 1, 0, 0, 0, 1, 0],
 [0, 0, 1, 1, 0, 0, 0, 1, 0],
 [0, 0, 1, 1, 0, 0, 0, 1, 0],
 [0, 0, 1, 1, 0, 0, 0, 1, 0],
 [0, 0, 1, 1, 0, 0, 0, 1, 0],
 [0, 0, 1, 1, 0, 0, 0, 1, 0],
 [0, 0, 1, 1, 0, 0, 0, 1, 0],
 [0, 0, 1, 1, 0, 0, 0, 1, 0],
 [0, 0, 1, 1, 0, 0, 0, 1, 0],
 [0, 0, 1, 1, 0, 0, 0, 1, 0],
 [0, 0, 1, 1, 0, 0, 0, 1, 0],
 [0, 0, 1, 1, 0, 0, 0, 1, 0],
 [0, 0, 1, 1, 0, 0, 0, 1, 0],
 [0, 0, 1, 1, 0, 0, 0, 1, 0],
 [0, 0, 1, 1, 0, 0, 0, 1, 0],
 [0, 0, 1, 1, 0, 0, 0, 1, 0],
 [0, 0, 1, 1, 0, 0, 0, 1, 0],
 [0, 0, 1, 1, 0, 0, 0, 1, 0],
 [0, 0, 1, 1, 0, 0, 0, 1, 0],
 [0, 0, 1,

In [70]:
BestValue

[618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5,
 618.5]